## Part1: Inroduction

### 1.1: Installation
TensorFlow is an open-source machine learning framework developed by Google. It allows developers to build and train machine learning models using a data flow graph that represents computation. TensorFlow is versatile and widely used for various machine learning tasks, including computer vision and natural language processing. It also includes a high-level API called Keras for quick and easy model building and training.

In [ ]:
%pip install tensorflow
import tensorflow as tf

### 1.2 Tensors
In machine learning and TensorFlow, a tensor is a multi-dimensional array of numerical values that represents input data, intermediate computations, and output data in a neural network. Each tensor has a data type (e.g., float32 or int64) and a shape, which defines the number of dimensions and the size of each dimension. Tensors can be created using the `tf.constant()` function and manipulated using a wide range of mathematical operations. Tensors are a critical component of machine learning and TensorFlow, providing a flexible and efficient way to represent and manipulate data in a neural network.

#### 1.2.1 0-d Tensors

In [3]:
city = tf.constant("Beijing", tf.string)
pi = tf.cos(tf.constant(3.1415926, tf.float32))

print("`city` is a {}-d tensor".format(tf.rank(city).numpy()))
print("`pi` is a {}-d tensor".format(tf.rank(pi).numpy()))

`city` is a 0-d tensor
`pi` is a 0-d tensor


#### 1.2.2 1-d Tensors
Vectors and lists can create 1-d Tensors

In [6]:
cities = tf.constant(["Beijing", "Shanghai", "Guangzhou"], tf.string)
fvrt_numbers = tf.constant([3.1415926, 2.71828], tf.float32)

print("`cities` is a {}-d tensor with shape {}".format(tf.rank(cities).numpy(), tf.shape(cities)))
print("`fvrt_numbers` is a {}-d tensor with shape {}".format(tf.rank(fvrt_numbers).numpy(), tf.shape(fvrt_numbers)))

`cities` is a 1-d tensor with shape [3]
`fvrt_numbers` is a 1-d tensor with shape [2]


#### 1.2.3 Higher Dimensional Tensors

You can create 2-d, 3-d 4-d and even higher order Tensors

In [7]:
# a 2-d tensor
matrix = tf.constant([[1, 2, 3], [4, 5, 6]], tf.int32)

# a 3-d tensor
data = tf.ones([10, 256, 256], tf.int32)

# a 4-d tensor
images = tf.zeros([10, 256, 256, 3], tf.int32)

print("matrix is a {}-d tensor with shape {}".format(tf.rank(matrix).numpy(), tf.shape(matrix)))
print("data is a {}-d tensor with shape {}".format(tf.rank(data).numpy(), tf.shape(data)))
print("images is a {}-d tensor with shape {}".format(tf.rank(images).numpy(), tf.shape(images)))

matrix is a 2-d tensor with shape [2 3]
data is a 3-d tensor with shape [ 10 256 256]
images is a 4-d tensor with shape [ 10 256 256   3]


### 1.3 Computations on Tensors
In TensorFlow, computations occur in tensors through a process called data flow graph computation. A data flow graph is a directed graph where the nodes represent mathematical operations and the edges represent the tensors that flow between them. When a TensorFlow program is executed, it first builds a data flow graph, which specifies the operations to be performed and the order in which they should be executed.

Once the data flow graph is constructed, TensorFlow performs a computation called a forward pass, which propagates the input data through the graph to produce an output. During the forward pass, each node in the graph performs its operation on the input tensors it receives and passes the output tensor(s) to the next node(s) in the graph. This process continues until the output tensor(s) are produced by the final node in the graph.

After the forward pass is completed, TensorFlow performs a backward pass, which calculates the gradients of the output tensor(s) with respect to the input tensor(s). These gradients are used in a process called backpropagation, which updates the weights of the neural network to minimize the loss function.

Overall, computations in TensorFlow occur in tensors through a data flow graph computation process that involves a forward pass and a backward pass. This process allows TensorFlow to efficiently perform complex mathematical operations on large datasets and train sophisticated machine learning models.

In [8]:
# create nodes in a graph
a = tf.constant(15, name="a")
b = tf.constant(61, name="b")

# add them
c1 = tf.add(a, b, name="c1")
c2 = a + b # TensorFlow overrides the "+" operation so that it is able to act on tensors
print(c1)
print(c2)

tf.Tensor(76, shape=(), dtype=int32)
tf.Tensor(76, shape=(), dtype=int32)


## Part2: Neural Networks in TF

### 2.1 Defining Neural Networks
TensorFlow provides a high level API `Keras` to define neural networks.

#### 2.1.1 Defining a custom Layer
This is defining a custom dense layer in TensorFlow using the `tf.keras.layers.Layer` class as a base. The `__init__` method initializes the number of output nodes for the layer. The `build` method creates the weights for the layer, which are the weight matrix `W` and bias vector `b`. The `call` method performs the forward pass of the layer, which involves matrix multiplication of the input `x` with the weight matrix `W`, adding the bias vector `b`, and applying the sigmoid activation function to the result.


In [12]:
class OurDenseLayer(tf.keras.layers.Layer):
    def __init__(self, n_output_nodes):
        super(OurDenseLayer, self).__init__()
        self.n_output_nodes = n_output_nodes

    def build(self, input_shape):
        d = int(input_shape[-1])
        self.W = self.add_weight("weight", shape=[d, self.n_output_nodes])
        self.b = self.add_weight("bias", shape=[1, self.n_output_nodes])

    def call(self, x):
        z = tf.matmul(x, self.W) + self.b
        y = tf.sigmoid(z)
        return y
    


# Since layer parameters are initialized lazily upon first call, you must
# call the layer at least once before using inspecting the weights
# (otherwise no weights will have been created)
layer = OurDenseLayer(3)
layer.build((1, 2))
print(layer.call(tf.ones([1, 2]).numpy()))
print(layer.trainable_variables)

tf.Tensor([[0.7276925  0.38845906 0.21887647]], shape=(1, 3), dtype=float32)
[<tf.Variable 'weight:0' shape=(2, 3) dtype=float32, numpy=
array([[ 0.39757   , -0.07050097, -0.2558784 ],
       [ 1.0046954 ,  0.12365389, -0.43444103]], dtype=float32)>, <tf.Variable 'bias:0' shape=(1, 3) dtype=float32, numpy=array([[-0.41931885, -0.5069469 , -0.58190644]], dtype=float32)>]


#### 2.1.2 Defining a neural network using Sequential API
The `tf.keras.Sequential` class provides a simple way to build a neural network with a single stack of layers. The `Sequential` constructor takes a list of layer instances, which are stacked together to build the neural network. The `add` method can also be used to add layers to the neural network. The `summary` method prints a summary of the neural network architecture.

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

n_output_nodes = 3

model = Sequential()

dense_layer = Dense(n_output_nodes, activation="sigmoid")
model.add(dense_layer)

In [18]:
x_input = tf.constant([[1, 2.]], shape=(1, 2))

model_output = model(x_input).numpy()
print(model_output)

[[0.30796766 0.26908135 0.59257346]]


#### 2.1.3 Defining a neural network using Subclassing API
The `tf.keras.Model` class provides a more flexible way to build a neural network by subclassing the `Model` class and defining the forward pass through the `call` method. The `__init__` method initializes the layers of the neural network. The `call` method performs the forward pass, which involves passing the input `x` through each layer in the neural network in order.

In [19]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense


class SubClassModel(tf.keras.Model):
    def __init__(self, n_output_nodes):
        super(SubClassModel, self).__init__()
        self.dense_layer = Dense(n_output_nodes, activation="sigmoid")

    def call(self, inputs):
        return self.dense_layer(inputs)

In [20]:
x_input = tf.constant([[1, 2.]], shape=(1, 2))

model = SubClassModel(3)

print(model.call(x_input))

tf.Tensor([[0.13677017 0.735966   0.48733085]], shape=(1, 3), dtype=float32)


## Part3: Automatic Derviative with TF

### 3.1 Automatic Differentiation
Automatic differentiation is a technique for numerically evaluating the derivative of a function. In TensorFlow, automatic differentiation is performed using a process called reverse-mode differentiation, which is also known as backpropagation. In reverse-mode differentiation, the derivatives are calculated starting from the output of the neural network and working backwards through the computational graph. This allows TensorFlow to efficiently calculate the gradients of the loss with respect to many variables by making a single pass through the computational graph.
`tf.GradientTape` is used to perform automatic differentiation in TensorFlow. To compute the gradient of a function, you can use `tf.GradientTape` in a `with` statement, and TensorFlow will automatically compute the gradient of any operations that occur inside the `with` statement.

In [21]:
# Gradient computation with GradientTape
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x * x

dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())

6.0
